<a href="https://colab.research.google.com/github/Vedantika2001/Calendar/blob/Expiry/NSE_Nifty_and_Bank_Nifty_Expiry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#NSE Nifty Weekly Expiry Updated

import pandas as pd
from datetime import datetime, timedelta

# Load the trading calendar CSV
df_calendar = pd.read_csv("test.csv")

# Convert 'Calendar Day' to datetime
df_calendar['Calendar Day'] = pd.to_datetime(df_calendar['Calendar Day'])

# Create a set of valid trading dates
trading_days = set(df_calendar[df_calendar['Trading Day'] == 1.0]['Calendar Day'])

# Helper function to get previous trading day
def get_previous_trading_day(date):
    while date not in trading_days:
        date -= timedelta(days=1)
    return date

# Expiry logic
launch_date = datetime(2019, 2, 11)
change_date = datetime(2025, 4, 4)
end_date = df_calendar['Calendar Day'].max()

expiry_data = []
current_date = launch_date

while current_date <= end_date:
    if current_date < change_date:
        weekday_target = 3  # Thursday
    else:
        weekday_target = 0  # Monday

    days_ahead = (weekday_target - current_date.weekday() + 7) % 7
    days_ahead = days_ahead if days_ahead else 7
    scheduled_expiry = current_date + timedelta(days=days_ahead)
    actual_expiry = get_previous_trading_day(scheduled_expiry)

    expiry_data.append({
        'Scheduled Expiry': scheduled_expiry.date(),
        'Actual Expiry': actual_expiry.date()
    })

    current_date = scheduled_expiry + timedelta(days=1)

# Create DataFrame of expiry dates
df_expiry = pd.DataFrame(expiry_data)

# Add expiry marker to main calendar
df_calendar['NSE Nifty Weekly Expiry'] = df_calendar['Calendar Day'].apply(
    lambda x: 1 if x.date() in df_expiry['Actual Expiry'].values else 0
)

# Save to CSV (optional)
df_calendar.to_csv("NSE1.csv", index=False)


In [ ]:
# NSE Nifty Monthly Expiry

import pandas as pd
from datetime import datetime, timedelta

# Load Data
df = pd.read_csv("test.csv")
df["Calendar Day"] = pd.to_datetime(df["Calendar Day"])

# Convert trading days into a set for fast lookup
trading_days = set(df[df["Trading Day"] == 1.0]["Calendar Day"].dt.date)

# Define important dates
monthly_expiry_start_date = datetime(2000, 6, 12).date()  # Nifty Monthly Expiry Start Date
expiry_shift_date = datetime(2025, 4, 4).date()  # Expiry shifts from Thursday to Monday

# Function to find last Thursday of the month
def get_last_thursday(year, month):
    if month == 12:
        first_day_next_month = datetime(year + 1, 1, 1)
    else:
        first_day_next_month = datetime(year, month + 1, 1)

    last_day = first_day_next_month - timedelta(days=1)

    while last_day.weekday() != 3:  # 3 represents Thursday
        last_day -= timedelta(days=1)

    return last_day.date()

# Function to find last Monday of the month
def get_last_monday(year, month):
    if month == 12:
        first_day_next_month = datetime(year + 1, 1, 1)
    else:
        first_day_next_month = datetime(year, month + 1, 1)

    last_day = first_day_next_month - timedelta(days=1)

    while last_day.weekday() != 0:  # 0 represents Monday
        last_day -= timedelta(days=1)

    return last_day.date()

# Function to determine monthly expiry status
def get_monthly_expiry_status(date):
    if date < monthly_expiry_start_date:
        return 0  # No expiry before June 2001

    if date < expiry_shift_date:
        # Before April 4, 2025, expiry is on the last Thursday
        expiry_date = get_last_thursday(date.year, date.month)
    else:
        # From April 4, 2025, expiry is on the last Monday
        expiry_date = get_last_monday(date.year, date.month)

    # Adjust expiry if it's a non-trading day
    while expiry_date not in trading_days:
        expiry_date -= timedelta(days=1)  # Shift to previous trading day

    return 1 if date == expiry_date else 0

# Apply function to dataset
df["NSE Nifty Monthly Expiry"] = df["Calendar Day"].dt.date.map(get_monthly_expiry_status)

# Save updated CSV
df.to_csv("updated_calendar_nifty_monthly10.csv", index=False)
print("✅ NSE Nifty Monthly Expiry column added successfully!")


In [ ]:
# NSE BankNifty Weekly Expiry

import pandas as pd
from datetime import datetime, timedelta

# Load the CSV file
df = pd.read_csv("Calendar - Sheet6.csv")

# Convert "Calendar Day" to datetime
df["Calendar Day"] = pd.to_datetime(df["Calendar Day"])

# Convert trading days to a set for fast lookup
trading_days = set(df[df["Trading Day"] == 1.0]["Calendar Day"].dt.date)

# Define the start and shift dates for expiry rules
weekly_expiry_start_date = datetime(2016, 5, 27).date()  # Nifty weekly expiry started
expiry_shift_date = datetime(2023, 9, 6).date()  # Expiry shifts from Thursday to Monday

# Function to determine expiry status
def get_expiry_status(date):
    if date < weekly_expiry_start_date:
        return 0  # No expiry before Feb 11, 2019

    if date < expiry_shift_date:  # Before April 4, 2025 → Thursday expiry
        weekday = date.weekday()
        if weekday == 3 and date in trading_days:  # Thursday
            return 1
        if weekday == 2 and (date + timedelta(days=1)) not in trading_days and date in trading_days:
            return 1  # Expiry shifts to Wednesday if Thursday is a holiday

    else:  # From April 4, 2025 → Monday expiry
        weekday = date.weekday()
        if weekday == 2 and date in trading_days:  # Monday
            return 1
        if weekday == 1 and (date + timedelta(days=1)) not in trading_days and date in trading_days:
            return 1  # Expiry shifts to Friday if Monday is a holiday

    return 0  # No expiry

# Apply function to dataset
df["NSE BankNifty Weekly Expiry"] = df["Calendar Day"].dt.date.map(get_expiry_status)

# Save updated CSV
df.to_csv("updated_calendar2.csv", index=False)


In [ ]:
# Bank Nifty Monthly Expiry

import pandas as pd
from datetime import datetime, timedelta

# 🔹 Load your calendar CSV
df = pd.read_csv("Calendar - Sheet6.csv")
df["Calendar Day"] = pd.to_datetime(df["Calendar Day"])
df["Trading Day"] = df["Trading Day"].astype(float)

# 🔹 Set of trading dates for quick lookup
trading_days = set(df[df["Trading Day"] == 1.0]["Calendar Day"].dt.date)

# 🔹 Function to get last weekday of the month
def get_last_weekday(year, month, weekday_target):
    """Returns the last 'weekday_target' of a given month"""
    last_day = datetime(year, month, 1) + pd.offsets.MonthEnd(0)
    while last_day.weekday() != weekday_target:
        last_day -= timedelta(days=1)
    return last_day.date()

# 🔹 Bank Nifty Monthly Expiry Detection
def is_banknifty_monthly_expiry(date):
    if date < datetime(2005, 6, 13).date():
        return 0  # Not launched yet

    year, month = date.year, date.month

    # Set expiry weekday based on policy change
    if datetime(2024, 3, 1).date() <= date <= datetime(2024, 12, 31).date():
        expiry_weekday = 2  # Wednesday
    else:
        expiry_weekday = 3  # Thursday

    # Get last weekday of that type
    last_weekday = get_last_weekday(year, month, expiry_weekday)

    # If it's not a trading day, move backward
    while last_weekday not in trading_days:
        last_weekday -= timedelta(days=1)

    return 1 if date == last_weekday else 0

# 🔹 Apply to dataset
df["Bank Nifty Monthly Expiry"] = df["Calendar Day"].dt.date.map(is_banknifty_monthly_expiry)

# 🔹 Save updated file
df.to_csv("updated_banknifty_expiry.csv", index=False)
print("✅ File saved with Bank Nifty Monthly Expiry marked!")
